## Load Reddit Comments Data into Parquet <a class="tocSkip">
This notebook loads the raw [Reddit comments dataset](http://academictorrents.com/details/85a5bd50e4c365f8df70240ffd4ecc7dec59912b) into a parquet file format. It does augment the data with several improved time columns, and the partitions the data by year/month/day. The file paths in this notebook should be modified for your system.

In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

import pandas as pd

pd.set_option('display.max_colwidth', -1)

spark = SparkSession\
        .builder\
        .appName("RedditLoadToParquet")\
        .getOrCreate()

In [2]:
reddit_raw = spark.read.json('qfs:///data/reddit/raw/*.bz2')

In [3]:
reddit_raw.printSchema()

root
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- num_reports: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- r

In [4]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

reddit_augmented = (
    reddit_raw
    .withColumn('created_utc', F.col('created_utc').cast(T.LongType()))
    .withColumn('created_date', F.from_unixtime('created_utc', 'yyyy-MM-dd'))
    .withColumn('year', F.from_unixtime('created_utc', 'yyyy'))
    .withColumn('month', F.from_unixtime('created_utc', 'MM'))
    .withColumn('day', F.from_unixtime('created_utc', 'dd'))
)

In [5]:
reddit_augmented.write.partitionBy(
    'year', 'month', 'day'
).parquet(
    'qfs:///data/reddit/processed',
    mode='overwrite'
)

In [6]:
reddit_augmented.limit(10).toPandas()

,approved_by,archived,author,author_flair_css_class,author_flair_text,banned_by,body,body_html,controversiality,created,...,score,score_hidden,subreddit,subreddit_id,ups,user_reports,created_date,year,month,day
0,None,True,adrianmonk,None,None,None,There's no way this guy is *really* Korean. He hasn't got the pathological self-deprecation right.,None,0,None,...,25,False,funny,t5_2qh33,25,None,2009-07-01,2009,07,01
1,None,True,Nesemulator,None,None,None,Lord of the Rings Online: has an absolutely awesome Sountrack.,None,0,None,...,2,False,gaming,t5_2qh03,2,None,2009-07-01,2009,07,01
2,None,True,adolfojp,None,None,None,The fuck it song in Spanish: http://www.imeem.com/people/4fSiML/music/QhKQd-up/la-banda-algarete-que-se-joda/,None,0,None,...,1,False,funny,t5_2qh33,1,None,2009-07-01,2009,07,01
3,None,True,thezoner,None,None,None,Ha! Same as hockey players. They must enjoy bloody knuckles.,None,0,None,...,1,False,videos,t5_2qh1e,1,None,2009-07-01,2009,07,01
4,None,True,ggrayskies,None,None,None,*You* are clever my friend.,None,0,None,...,2,False,pics,t5_2qh0u,2,None,2009-07-01,2009,07,01
5,None,True,nikniuq,None,None,None,Does it *have* to be human?,None,0,None,...,1,False,reddit.com,t5_6,1,None,2009-07-01,2009,07,01
6,None,True,[deleted],None,None,None,"I've been lurking reddit for 3 years. Didn't register because I figured an e-mail address was required. I didn't even bother to check. \n\nAfter seeing this, I have an account! Woo hoo!",None,0,None,...,3,False,technology,t5_2qh16,3,None,2009-07-01,2009,07,01
7,None,True,Redmoons,None,None,None,"This one time, my friends and I got some m80's and lit them off.\r\n\r\nMy step-mom's lap dog thought the sparkling end of the fuse to one of them was really interesting, and picked up the m80 like it was a stick.\r\n\r\nThe m80 blew it's lower jaw off. \r\n\r\nI hated that dog.\r\n\r\nlol.",None,0,None,...,-1,False,AskReddit,t5_2qh1i,-1,None,2009-07-01,2009,07,01
8,None,True,[deleted],None,None,None,The Chrono Cross soundtrack is my all time favorite.,None,0,None,...,1,False,gaming,t5_2qh03,1,None,2009-07-01,2009,07,01
9,None,True,larva,None,None,None,"Well, it's pretty good, but the famous [Epic Thread](http://www.reddit.com/comments/6nz1k/got_six_weeks_try_the_hundred_push_ups_training/c04ehte), is way smarter and funnier, in my opinion.",None,0,None,...,9,False,reddit.com,t5_6,9,None,2009-07-01,2009,07,01
